In [0]:
%run "/Workspace/Vehicle_Project/common"

### Reading from bronze raw_Roads

In [0]:
dbutils.widgets.text(name = "environment", defaultValue = "", label = "please type environment")
current_environment = dbutils.widgets.get("environment")

In [0]:
def read_bronzeRoadsTable(env):
    print("Reading Bronze Roads Data:  '", end='')
    df_bronzeRoads =( spark.readStream
                          .table(f"`{env}_catalog`.`bronze`.`raw_roads`")
                          )
    print(f'Reading {env}_catalog`.`bronze`.`raw_traffic` successfully read!!')
    return df_bronzeRoads


In [0]:
df_roads = read_bronzeRoadsTable(current_environment)

## creating road_category_name column


In [0]:
def road_Category(df):
    print('Creating Road Category Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Cat = df.withColumn("Road_Category_Name",
                  when(col('Road_Category') == 'TA', 'Class A Trunk Road')
                  .when(col('Road_Category') == 'TM', 'Class A Trunk Motor')
                   .when(col('Road_Category') == 'PA','Class A Principal road')
                    .when(col('Road_Category') == 'PM','Class A Principal Motorway')
                    .when(col('Road_Category') == 'M','Class B road')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Cat

## creating road_type column


In [0]:
def road_Type(df):
    print('Creating Road Type Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Type = df.withColumn("Road_Type",
                  when(col('Road_Category_Name').like('%Class A%'),'Major')
                  .when(col('Road_Category_Name').like('%Class B%'),'Minor')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Type

## Writing siver roads to silver schema

In [0]:
def write_Roads_SilverTable(StreamingDF,environment):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint_location+ "/SilverRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`silver`.`silver_roads`"))
    
    write_StreamSilver_R.awaitTermination()
    print(f'Writing `{environment}_catalog`.`silver`.`silver_roads` Success!')

## calling function

In [0]:

## To remove duplicate rows
df_dups = remove_dups(df_roads)

## To replace anu NULL values
All_columns_bronzeroads = df_dups.schema.names
df_nulls = handle_NULLS(df_dups, All_columns_bronzeroads)

## creating Road_category_name
df_roadCat = road_Category(df_nulls)

## writing data to silver_roads table
write_Roads_SilverTable(df_roadCat,current_environment)

In [0]:
%sql
SELECT * FROM `dev_catalog`.`silver`.`silver_roads`